In [ ]:
# default_exp files

# files
> Easily upload, download, and manage files on Server.

In [ ]:
# export
import requests

from yx_motor.api import API


class Files:
    "Class for handling AAH files endpoints."

    def __init__(self, api: API):
        self.api = api
        self.base_endpoint = "files/"
        self.valid_conflict_actions = ["MERGE", "CREATE_COPY"]

    def download_file(self, file_uuid: str, download_path: str, version: int = None):
        response = self.api.get(
            url=f"{self.base_endpoint}content",
            params={"id": file_uuid, "version": version},
        )
        with open(download_path, "wb") as f:
            f.write(response.content)
        return response

    def upload_file(
        self,
        filename: str,
        upload_path: str,
        description: str = None,
        conflict_action: str = "CREATE_COPY",
    ):
        if conflict_action:
            self.validate_conflict_action(conflict_action)

        upload_headers = {
            "Content-Type": "application/json",
            "Accept": "*/*",
            "Accept-Language": "en-US,en;q=0.5",
            "Accept-Encoding": "gzip,deflate",
            "path": upload_path,
            "description": description,
            "conflict_action": conflict_action,
        }

        with open(filename, "rb") as f:
            blob = f.read()
        response = self.api.post(
            url=f"{self.base_endpoint}", data=blob, non_default_headers=upload_headers
        )
        return response

    def update_file(self):
        # This seems to update metainfo, may not be needed for MVP
        # TODO: MVP
        pass

    def get_file_versions(self, file_uuid: str):
        response = self.api.get(
            url=f"{self.base_endpoint}versions/", params={"fileUuid": file_uuid}
        )
        return response

    def delete_file(self, asset_path: str, hard=False):
        payload = {"assetPaths": [f"{asset_path}"]}

        if hard == True:
            targeturl = "remove"
        else:
            targeturl = "softDelete"

        response = self.api.post(url=f"{self.base_endpoint}{targeturl}", json=payload)

        return response

    def move_file(
        self,
        source_path: str,
        target_path: str,
        move_type="moveFiles",
        versions_action="ALL_VERSIONS",
        conflicts_action="SKIP",
    ):
        # TODO discuss move_type, move or copy
        payload = {
            "assets": [
                {"sourcePath": f"{source_path}", "targetPath": f"{target_path}"}
            ],
            "versionsAction": f"{versions_action}",
            "conflictsAction": f"{conflicts_action}",
        }

        response = self.api.post(url=f"{self.base_endpoint}{move_type}", json=payload)
        return response

    def restore_deleted_file(self, asset_path: str = None, asset_id: str = None):
        asset_paths = {}
        asset_ids = {}
        if asset_path:
            asset_paths = {"assetPaths": [f"{asset_path}"]}
        if asset_id:
            asset_ids = {"assetIds": [f"{asset_id}"]}
        payload = {**asset_paths, **asset_ids}
        # Does onlyDescendants have other values besides True?
        response = self.api.post(
            url=f"{self.base_endpoint}restoreDeleted", json=payload
        )
        return response

    def validate_conflict_action(self, conflict_action: str):
        if conflict_action not in self.valid_conflict_actions:
            raise ValueError(
                f"Specified conflict action must be one of {self.valid_conflict_actions}"
            )
        else:
            pass

In [ ]:
#hide
# just removing the insecure warning for now
# TODO: Secure requests and remove this code
import urllib3
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

## Examples

**Note**:
The files commands are intended to be called by users through the Client class.  Below, you will see that to accommodate this, we are creating the files object the same way the Client class does, by passing in an authenticated API object.  

In normal practice, a user will simply create a client object, and be able to call `client.files.download_file('xxxxxx-xxxxx')`.

However, for this documentation, we are calling the files object directly to better demonstrate the Files class in an isolated fashion.

### Download Files Example

In [ ]:
from nbdev.showdoc import *
show_doc(Files.download_file)

<h4 id="Files.download_file" class="doc_header"><code>Files.download_file</code><a href="__main__.py#L15" class="source_link" style="float:right">[source]</a></h4>

> <code>Files.download_file</code>(**`file_uuid`**:`str`, **`download_path`**:`str`, **`version`**:`int`=*`None`*)



The download_file method allows a user to pass in the unique identifier for the file, along with the path they wish to download the file to on the client machine.  The file will be downloaded and saved to the path specified.

**Arguments**:
- file_uuid: The VFS unique identifier for the file the user wishes to download.
- download_path: The path on the user's local file system which they wish the file to be persisted to once it is downloaded.
- version: Optional.  The version of the VFS asset the user wishes to donwload.  Defaults to latest version if not specified.  TODO: Need to double check this statement.

In [ ]:
#hide

# Unit test code for download files

from unittest.mock import Mock

from yx_motor.tests.utils.unit_test_helpers import (
    files_test_pickles,
    unpickle_object
)

api = Mock()
api.get = Mock()
api.get.return_value = unpickle_object(files_test_pickles.download_file)

In [ ]:
# Initialize the files object the same way the Client object does.
files = Files(api)

Below is the unique identifier for the file we wish to download from the Alteryx Analytics Hub's Virtual File System (VFS).

In [ ]:
asset_id = "9b75b3d8-56c1-4ce6-bfe1-9aabaea1e65f"

Next, we will trigger the method.  To better illustrate what the download_file method does, we are going to capture the response.  It is important to note that this is not necessary.  Simply calling the `files.download_file` command is all that is necessary to successfully download a file to the specified path.

Next, let's trigger the download_file command.

In [ ]:
response = files.download_file(file_uuid=asset_id,
                               download_path=f"example_downloads/some_name.yxmd")

As you can see, the workflow has been downloaded to the specified location.

Now, let's inspect the response object.

In [ ]:
response.content # The requested file is attached to the response as a binary object.

b'<?xml version="1.0"?>\r\n<AlteryxDocument yxmdVer="2020.1">\r\n  <Nodes>\r\n    <Node ToolID="1">\r\n      <GuiSettings Plugin="AlteryxBasePluginsGui.TextInput.TextInput">\r\n        <Position x="222" y="174" />\r\n      </GuiSettings>\r\n      <Properties>\r\n        <Configuration>\r\n          <NumRows value="1" />\r\n          <Fields>\r\n            <Field name="Field1" />\r\n          </Fields>\r\n          <Data>\r\n            <r>\r\n              <c>WorkflowA</c>\r\n            </r>\r\n          </Data>\r\n        </Configuration>\r\n        <Annotation DisplayMode="0">\r\n          <Name />\r\n          <DefaultAnnotationText />\r\n          <Left value="False" />\r\n        </Annotation>\r\n      </Properties>\r\n      <EngineSettings EngineDll="AlteryxBasePluginsEngine.dll" EngineDllEntryPoint="AlteryxTextInput" />\r\n    </Node>\r\n    <Node ToolID="2">\r\n      <GuiSettings Plugin="AlteryxBasePluginsGui.DbFileOutput.DbFileOutput">\r\n        <Position x="378" y="174" />

Here, we see that the binary object is indeed an Alteryx workflow.

### Upload Files Example

In [ ]:
from nbdev.showdoc import *
show_doc(Files.upload_file)

<h4 id="Files.upload_file" class="doc_header"><code>Files.upload_file</code><a href="__main__.py#L24" class="source_link" style="float:right">[source]</a></h4>

> <code>Files.upload_file</code>(**`filename`**:`str`, **`upload_path`**:`str`, **`description`**:`str`=*`None`*, **`conflict_action`**:`str`=*`'CREATE_COPY'`*)



The upload_file method allows a user to upload a file from their local filesytem to the Alteryx Analytics Hub's Virtual File System (VFS).

TODO: Need to rename the filename argument to file_path to avoid confusion.

**Arguments**:
- filename: Path to the file on the user's local file system
- upload_path: The path on the AAH VFS the user wishes the file to be uploaded to.
- description: Optional.  Description the user wants to be attached to the file in the VFS.
- conflict_action: Defaults to 'CREATE_COPY'.  One of 'CREATE_COPY'| 'MERGE'
    - CREATE_COPY: If file already exists at specified path, a copy is created.
    - MERGE: If file already exists, new version is created at same location.


In [ ]:
#hide

# Unit test code for download files

from unittest.mock import Mock

from yx_motor.tests.utils.unit_test_helpers import (
    files_test_pickles,
    unpickle_object
)

api = Mock()
api.post = Mock()
api.post.return_value = unpickle_object(files_test_pickles.upload_file)

In [ ]:
# Initialize the files object the same way the Client object does.
files = Files(api)

#### Simple Upload Example

Let's upload a yxzp file in our local filesystem to a location on Alteryx Analytics Hub's VFS, specified via the upload_path parameter.

In [ ]:
response = files.upload_file(filename=r"example_downloads/some_name.yxmd",
                             upload_path="/Workspaces/Public/example_1.yxzp")

In [ ]:
response.json()

{'folderType': None,
 'comment': None,
 'maxVersion': 1,
 'links': None,
 'created': '2020-08-04T22:55:02.066Z',
 'versionCreated': '2020-08-04T22:55:02.066Z',
 'metaHash': 'd214d65ab55a2a1c73de7c90fcc1aecfc28dd88dbec71cdeaeb3ffb1ef170053',
 'isHidden': False,
 'assetCategory': 'WORKFLOW_APP',
 'extension': 'yxzp',
 'onlyOwnerShares': False,
 'onlySiteAdminShares': False,
 'contentId': 'ed3146c9-8cf5-4099-8008-c3657b29e3a6',
 'contentHash': 'b7960c08c75e5e4e2bcf5fa32582eee75a07560554c6993c603759a8a2ce2f5f',
 'contentSize': 2176,
 'entryOwner': {'avatar': None,
  'email': 'apiuser@example.com',
  'firstName': 'Aaaaaaaaaaa',
  'id': 'd57f3054-8eeb-4993-b021-cc7c6764d27c',
  'lastName': 'ApiTest',
  'name': 'Aaaaaaaaaaa ApiTest',
  'userName': 'apiuser'},
 'metadata': {'yxType': 'WORKFLOW_APP'},
 'fileName': 'example_1.yxzp',
 'inherits': True,
 'path': '/Workspaces/Public/example_1.yxzp',
 'version': 1,
 'assignedPermissionsDateUpdated': None,
 'location': '',
 'uuid': '5ee160b4-51fa-4ba

Above, we can see that the response shows the file was successfully uploaded to the upload path we specified.  

#### Upload with duplicate file example

What happens if we upload the same file twice?  Let's run the same line of code a second time to see.

In [ ]:
#hide

# Unit test code for download files

from unittest.mock import Mock

from yx_motor.tests.utils.unit_test_helpers import (
    files_test_pickles,
    unpickle_object
)

api = Mock()
api.post = Mock()
api.post.return_value = unpickle_object(files_test_pickles.upload_duplicate_file)

In [ ]:
response = files.upload_file(filename=r"example_downloads/some_name.yxmd",
                             upload_path="/Workspaces/Public/example_1.yxzp")

In [ ]:
response.json()

{'folderType': None,
 'comment': None,
 'maxVersion': 1,
 'links': None,
 'created': '2020-08-04T22:55:02.066Z',
 'versionCreated': '2020-08-04T22:55:02.066Z',
 'metaHash': 'd214d65ab55a2a1c73de7c90fcc1aecfc28dd88dbec71cdeaeb3ffb1ef170053',
 'isHidden': False,
 'assetCategory': 'WORKFLOW_APP',
 'extension': 'yxzp',
 'onlyOwnerShares': False,
 'onlySiteAdminShares': False,
 'contentId': 'ed3146c9-8cf5-4099-8008-c3657b29e3a6',
 'contentHash': 'b7960c08c75e5e4e2bcf5fa32582eee75a07560554c6993c603759a8a2ce2f5f',
 'contentSize': 2176,
 'entryOwner': {'avatar': None,
  'email': 'apiuser@example.com',
  'firstName': 'Aaaaaaaaaaa',
  'id': 'd57f3054-8eeb-4993-b021-cc7c6764d27c',
  'lastName': 'ApiTest',
  'name': 'Aaaaaaaaaaa ApiTest',
  'userName': 'apiuser'},
 'metadata': {'yxType': 'WORKFLOW_APP'},
 'fileName': 'example_1.yxzp',
 'inherits': True,
 'path': '/Workspaces/Public/example_1.yxzp',
 'version': 1,
 'assignedPermissionsDateUpdated': None,
 'location': '',
 'uuid': '5ee160b4-51fa-4ba

If you look above, you see that a copy of the file was created in the same location, but now the file is named example_1(1).yxzp.  

NOTE: It WAS named that in the payload.  Update at noon on 4/27/20 seems to have broken this, so that the UI shows the example_1(1) name, but the vfs path appears to now have a uid appended to it.

This is because the default conflict action is 'CREATE_COPY'.  

#### Upload a file with MERGE command

Let's now repeat the call, but this time let's specify 'MERGE' as the conflict action.  Also, we're changing the source of the file to a larger one, to ensure that the contentSize property in the JSON response proves that the file has been updated.

In [ ]:
#hide

# Unit test code for download files

from unittest.mock import Mock

from yx_motor.tests.utils.unit_test_helpers import (
    files_test_pickles,
    unpickle_object
)

api = Mock()
api.post = Mock()
api.post.return_value = unpickle_object(files_test_pickles.upload_merge_file)

files = Files(api)

In [ ]:
response = files.upload_file(filename=r"example_downloads/some_name.yxmd",
                             upload_path="/Workspaces/Public/example_1.yxzp",
                             conflict_action='MERGE')

In [ ]:
response.json()

{'folderType': None,
 'comment': None,
 'maxVersion': 2,
 'links': None,
 'created': '2020-08-04T22:55:02.066Z',
 'versionCreated': '2020-08-04T22:56:49.425Z',
 'metaHash': '54cdf04ba27078ce1a537f264048e53ee6ade4dcbfcc6bc483c58861909cf184',
 'isHidden': False,
 'assetCategory': 'WORKFLOW_APP',
 'extension': 'yxzp',
 'onlyOwnerShares': False,
 'onlySiteAdminShares': False,
 'contentId': '03cfebd9-a031-4b98-a1ec-7176224e0762',
 'contentHash': 'c1d63578a41c19764af294616acac0f84c6aa40781496b643ebcbf644b79b642',
 'contentSize': 4055,
 'entryOwner': {'avatar': None,
  'email': 'apiuser@example.com',
  'firstName': 'Aaaaaaaaaaa',
  'id': 'd57f3054-8eeb-4993-b021-cc7c6764d27c',
  'lastName': 'ApiTest',
  'name': 'Aaaaaaaaaaa ApiTest',
  'userName': 'apiuser'},
 'metadata': {'tools': [],
  'yxType': 'WORKFLOW',
  'vfsInputs': {},
  'dependencies': {},
  'parsedHashes': [],
  'workflowInfo': {'isE2': False,
   'modules': ['Magic-8-Ball.yxmd'],
   'otherFiles': []}},
 'fileName': 'example_1.yxzp'

In the payload above, you can see that the version of the file is now 2, and that it's contentSize property is now 4055, bigger than the original 2176 value.

This demonstrates that the file has indeed been updated in place, with a versioned copy.  

This MERGE command has a very powerful property:

It uses a "content hash" on the file being uploaded to detect if the file has actually changed.  If we run the above command again, with the same file, we will see that the file's version remains the same.

In [ ]:
# Extract the file uuid from the response to demonstrate the get versions module later.
uploaded_file_uuid = response.json()['uuid']

In [ ]:
from nbdev.showdoc import *
show_doc(Files.get_file_versions)

<h4 id="Files.get_file_versions" class="doc_header"><code>Files.get_file_versions</code><a href="__main__.py#L56" class="source_link" style="float:right">[source]</a></h4>

> <code>Files.get_file_versions</code>(**`file_uuid`**:`str`)



The get_file_versions method returns a list of all of the versions for a given VFS asset.

**Arguments**:
- file_uuid: VFS Asset unique identifier for the VFS file you want to get versions for.

In [ ]:
#hide

# Unit test code for download files

from unittest.mock import Mock

from yx_motor.tests.utils.unit_test_helpers import (
    files_test_pickles,
    unpickle_object
)

api = Mock()
api.get = Mock()
api.get.return_value = unpickle_object(files_test_pickles.get_file_versions)

files = Files(api)

In [ ]:
response = files.get_file_versions(file_uuid=uploaded_file_uuid)

In [ ]:
response.json()

[{'version': 2,
  'metadata': {'tools': [],
   'yxType': 'WORKFLOW',
   'vfsInputs': {},
   'dependencies': {},
   'parsedHashes': [],
   'workflowInfo': {'isE2': False,
    'modules': ['Magic-8-Ball.yxmd'],
    'otherFiles': []}},
  'comment': None,
  'creator': {'avatar': None,
   'email': 'apiuser@example.com',
   'firstName': 'Aaaaaaaaaaa',
   'id': 'd57f3054-8eeb-4993-b021-cc7c6764d27c',
   'lastName': 'ApiTest',
   'name': 'Aaaaaaaaaaa ApiTest'},
  'owner': {'avatar': None,
   'email': 'apiuser@example.com',
   'firstName': 'Aaaaaaaaaaa',
   'id': 'd57f3054-8eeb-4993-b021-cc7c6764d27c',
   'lastName': 'ApiTest',
   'name': 'Aaaaaaaaaaa ApiTest'},
  'createdAt': '2020-08-04T22:57:59.824Z'},
 {'version': 1,
  'metadata': {'tools': [],
   'yxType': 'WORKFLOW',
   'vfsInputs': {},
   'dependencies': {},
   'parsedHashes': [],
   'workflowInfo': {'isE2': False,
    'modules': ['Magic-8-Ball.yxmd'],
    'otherFiles': []}},
  'comment': None,
  'creator': {'avatar': None,
   'email': 'a

In [ ]:
from nbdev.showdoc import *
show_doc(Files.delete_file)

<h4 id="Files.delete_file" class="doc_header"><code>Files.delete_file</code><a href="__main__.py#L62" class="source_link" style="float:right">[source]</a></h4>

> <code>Files.delete_file</code>(**`asset_path`**:`str`, **`hard`**=*`False`*)



The delete_file method deletes a given asset located at the specified asset path in the VFS. Optionally, the user can specify whether the delete is permanent (hard=True) or reversable (hard=False).  

**Arguments**:
- asset_path: The VFS path of the asset the user wants to delete.
- hard: (Optional, defaults to False) If true, specifies a hard (permanent) delete. If false, the deleted asset can be restored.

In [ ]:
#hide

# Unit test code for download files

from unittest.mock import Mock

from yx_motor.tests.utils.unit_test_helpers import (
    files_test_pickles,
    unpickle_object
)

api = Mock()
api.post = Mock()
api.post.return_value = unpickle_object(files_test_pickles.delete_file)

files = Files(api)

In [ ]:
response = files.delete_file(asset_path=r'/Workspaces/Public/example_1.yxzp', 
                             hard=True)

In [ ]:
response.json()

{'removed': [{'assetPath': '/Workspaces/Public/example_1.yxzp',
   'assetId': '5ee160b4-51fa-4baf-8d36-145be6a247c0',
   'contentSize': 4055,
   'version': 2,
   'originalLocation': None,
   'folderType': None}],
 'failed': []}

In [ ]:
from nbdev.showdoc import *
show_doc(Files.move_file)

<h4 id="Files.move_file" class="doc_header"><code>Files.move_file</code><a href="__main__.py#L74" class="source_link" style="float:right">[source]</a></h4>

> <code>Files.move_file</code>(**`source_path`**:`str`, **`target_path`**:`str`, **`move_type`**=*`'moveFiles'`*, **`versions_action`**=*`'ALL_VERSIONS'`*, **`conflicts_action`**=*`'SKIP'`*)



The move_file method moves a given asset from the source_path in the VFS to the target_path. Optionally, the user can specify the move_type, versions_action, or conflicts_action.

**Arguments**:
- source_path: The VFS path of the asset the user wants to move.
- target_path: The destination path in the VFS where the user wants the file moved to.
- move_type: (optional, defaults to moveFiles) If 'moveFiles', moves the file from source_path to target_path. If 'copy', creates a copy of the file in the target_path
- versions_action: (optional, defaults to ALL_VERSIONS) specifies if version history should be moved along with the file
- conflicts_action: (optional, defaults to SKIP) specifies what should happen if a file with the same name already exists in the target_path. options are SKIP (no action taken), FAIL_OPERATION (return an error), MERGE (update the file in target_path with the new file from source_path).

In [ ]:
#hide

# Unit test code for download files

from unittest.mock import Mock

from yx_motor.tests.utils.unit_test_helpers import (
    files_test_pickles,
    unpickle_object
)

api = Mock()
api.post = Mock()
api.post.return_value = unpickle_object(files_test_pickles.move_file)

files = Files(api)

### Simple move example

Lets try using a move command to rename a file

In [ ]:
response = files.move_file(
    r"/Workspaces/Public/move_file_example.yxmd",
    r"/Workspaces/Public/move_file_destination.yxmd"
)

In [ ]:
response.json()

[{'assetPath': '/Workspaces/Public/move_file_example.yxmd',
  'successMoved': [{'version': 1,
    'folderType': None,
    'targetId': '037e00a0-16ce-4a2a-a953-704c34772477',
    'sourceId': '037e00a0-16ce-4a2a-a953-704c34772477',
    'involvedVersions': 1,
    'isMergeOperation': False,
    'sourcePath': '/Workspaces/Public/move_file_example.yxmd',
    'targetPath': '/Workspaces/Public/move_file_destination.yxmd'}]}]

### Simple copy example

Now, lets try copying a file.

In [ ]:
#hide

# Unit test code for download files

from unittest.mock import Mock

from yx_motor.tests.utils.unit_test_helpers import (
    files_test_pickles,
    unpickle_object
)

api = Mock()
api.post = Mock()
api.post.return_value = unpickle_object(files_test_pickles.copy_file)

files = Files(api)

In [ ]:
response = files.move_file(
    r"/Workspaces/Public/move_file_example.yxmd",
    r"/Workspaces/Public/copied_file.yxmd",
    "copy"
)

In [ ]:
response.json()

[{'assetPath': '/Workspaces/Public/move_file_example.yxmd',
  'successCopied': [{'targetPath': '/Workspaces/Public/copied_file.yxmd',
    'sourcePath': '/Workspaces/Public/move_file_example.yxmd',
    'version': 1,
    'folderType': None,
    'sourceId': '27be134c-423f-4040-8262-98359be4d2ae',
    'involvedVersions': 1,
    'isMergeOperation': False,
    'targetId': 'fbb72c87-0bcb-4179-bce8-d4149a17e68f'}]}]

as you can see, in the response, there is a successCopied json object which contains information on the new file, including the targetId (the uuid of the new file).

In [ ]:
from nbdev.showdoc import *
show_doc(Files.restore_deleted_file)

<h4 id="Files.restore_deleted_file" class="doc_header"><code>Files.restore_deleted_file</code><a href="__main__.py#L94" class="source_link" style="float:right">[source]</a></h4>

> <code>Files.restore_deleted_file</code>(**`asset_path`**:`str`=*`None`*, **`asset_id`**:`str`=*`None`*)



The restore_deleted_file method restores a specified asset. it requires either an asset_path or an asset_id. If both are specified, the asset_id is used. This can only be used to restore assets which were not hard deleted

**Arguments**:
- asset_path: (Optional) the 'trashPath' to the deleted asset including filename
- asset_id: (Optional) the UUID of the deleted asset

In [ ]:
#hide

# Unit test code for download files

from unittest.mock import Mock

from yx_motor.tests.utils.unit_test_helpers import (
    files_test_pickles,
    unpickle_object
)

api = Mock()
api.post = Mock()
api.post.return_value = unpickle_object(files_test_pickles.restore_deleted)

files = Files(api)

### Restore deleted file by uuid

here we will restore a deleted file by pointing to a uuid

In [ ]:
response = files.restore_deleted_file(asset_id='9b75b3d8-56c1-4ce6-bfe1-9aabaea1e65f')

In [ ]:
response.json()

{'failed': [],
 'restored': [{'assetPath': '/Trash/delete_file_example.yxmd',
   'successRestored': [{'trashPath': '/Trash/delete_file_example.yxmd',
     'assetId': '4852b7ce-e84b-4cbc-9f8b-24b304dc4af4',
     'folderType': None,
     'version': 1,
     'restorePath': '/Workspaces/Public/delete_file_example.yxmd'}]}]}

### Restore deleted file by asset_path

here we will restore a deleted file using the 'trashPath' provided when an asset is deleted.

In [ ]:
#hide

# Unit test code for download files

from unittest.mock import Mock

from yx_motor.tests.utils.unit_test_helpers import (
    files_test_pickles,
    unpickle_object
)

api = Mock()
api.post = Mock()
api.post.return_value = unpickle_object(files_test_pickles.restore_deleted_by_path)

files = Files(api)

In [ ]:
response = files.restore_deleted_file(asset_path=r'/Trash/delete_file_example.yxmd')

In [ ]:
response.json()

{'failed': [],
 'restored': [{'assetPath': '/Trash/delete_file_example.yxmd',
   'successRestored': [{'trashPath': '/Trash/delete_file_example.yxmd',
     'assetId': '4852b7ce-e84b-4cbc-9f8b-24b304dc4af4',
     'folderType': None,
     'version': 1,
     'restorePath': '/Workspaces/Public/delete_file_example.yxmd'}]}]}